# Tutorial for generating reliability field maps and gradient flow vectors:

## Reliability Field maps:  
Field maps show the contribution of the intra (x-axis) and inter (y-axis) variation to the ICC in a two dimensional histogram.

The field maps can be created using any measurable variable, but we focus on edges from a functional connectivity matrix. For a given edge, we plot the intra- and inter-variation on the x- and y-axis respectively revealing ICC (intra- and between-variation) via diagonal lines originating from point 0,0 in a rainbow-like manner. 

## Gradient flow vectors:
Built upon the variability field map, we also utilize the variability gradient flow map (GFM) for assessing the relative impact of reliability optimizations focused on one form of variation versus the other, depending on the current balance of the intra- and inter-individual variations. This allows us to identify how and where the difference in variability between two conditions changes in an optimal or sub-optimal direction towards the highest 

The gradient vector on the field map represents the optimal direction for maximal ICC change on the respective axes of individual variability. For a given ICC change, we normalized its gradient by the contributions of individual variability as compared to the optimal direction


#### Import functions needed for field maps and gradient flow vectors

In [1]:
%matplotlib inline
import os,sys
import cifti
import glob
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from nilearn import surface,plotting
from matplotlib.colors import LinearSegmentedColormap
sys.path.append('../code')
from gradient_flow_vectors import *
from reliability_field_maps import *
from variability_utils import *

/anaconda2/envs/py36/lib/python3.6/site-packages/nilearn/datasets/__init__.py:90: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)


#### Load surfaces to plot ICC differences and gradient flow vectors on the surface

* Put prior to surface plots

In [2]:
# Conte surface
lsurf = surface.load_surf_mesh('../misc/surfaces/Conte69.L.very_inflated.10k_fs_LR.surf.gii')
rsurf = surface.load_surf_mesh('../misc/surfaces/Conte69.R.very_inflated.10k_fs_LR.surf.gii')

# Surface plot background
bg = nib.load('../misc/surfaces/100206.sulc.10k_fs_LR.dscalar.nii').get_fdata()

# Example file to save surface plots
plabel = '../misc/Glasser2016_labels/HCP_MMP_P210_10k.dlabel.nii'
glasserlabel,(ax1,ax2) = cifti.read(plabel)
numverts = glasserlabel.shape[1]

#### Load colormaps and set gradient flow vector options:

In [3]:
# Colormaps
# rvb = vector_cmap() # Gradient flow vector colormap
rvb = icc_gradient_flow_cmap()
# yeo_colors = get_yeo_colors() # Yeo 7 network colors in RGB (turned into global variable)
allparcels = get_yeo_parcels() # Glasser 360 parcels to Yeo 7 network assignments

# Vector plot options:
outpath = False
vector_type = 'norm_0' # raw, norm, norm_0
alpha = 1 # plot option
# savefolder = '../figures'

## Data used:
 * Functional
     * Every edge from the upper triangle of a 360 x 360 functional connectivity matrix was used as input for the ICC model.
         * N<sub>subs</sub> x edges from upper triangle of functional connectivity matrix
     * The output arrays generated from the ICC model along with our specific data dictionary keys are listed below: 
         * ICC ('icc')
         * raw intra-individual variation ('raww')
         * raw inter-individual variation ('rawb')
         * total variation ('vartotal')
         * mask of failed edges ('totmask')
     * Also included are masked arrays which exclude failed edges. 
         * 'icc_masked', 'raww_masked', 'rawb_masked', 'vartotal_masked', 'totmask_masked'
 * Behavioral
     * N<sub>subs</sub> x 68 behavioral measures “Cognition”, “Emotion”, “Personality” (check “Motor” and “Sensory”)

## Remove masked and unused data
## Change masking to binary
## data as csv

In [ ]:
# Load Data:
data = np.load('../tutorial/example_data/tutorial_data.npy',allow_pickle=True).item()
tasks = [task for task in data.keys()]
print('Task conditions in data variable: %s' % tasks)
print('Dictionary keys within each task condition: %s' % [data_keys for data_keys in data[tasks[0]].keys()])

Task conditions in data variable: ['REST_nogsr', 'REST_gsr']
Dictionary keys within each task condition: ['rawb', 'raww', 'icc', 'vartotal', 'totmask', 'totmask_cortex', 'raww_masked', 'rawb_masked', 'icc_masked', 'vartotal_masked']


#### First obtain descriptives on GSR and no GSR ICC:

In [ ]:
for task in data.keys():
    print('%s ICC mean: %s +/- %s' % (task,np.mean(data[task]['icc_masked']),np.std(data[task]['icc_masked'])))
    print('%s Inter-individual variation mean: %s +/- %s' % (task,np.mean(data[task]['rawb_masked']),np.std(data[task]['rawb_masked'])))
    print('%s Intra-individual variation mean: %s +/- %s\n ' % (task,np.mean(data[task]['raww_masked']),np.std(data[task]['raww_masked'])))

    

REST_nogsr ICC mean: 0.3269461615406128 +/- 0.16618663921606122
REST_nogsr Inter-individual variation mean: 0.0064448203904029985 +/- 0.004434554694389156
REST_nogsr Intra-individual variation mean: 0.012462686539650639 +/- 0.004424689651843456
 
REST_gsr ICC mean: 0.33925891966189664 +/- 0.17487204079738128
REST_gsr Inter-individual variation mean: 0.00884100229180252 +/- 0.006779597231439544
REST_gsr Intra-individual variation mean: 0.015227109999291203 +/- 0.005133726686205739
 


Resting-state scans processed with GSR have greater mean ICC (0.34 +/- 0.17) compared to scans processed without GSR (0.33 +/- 0.17)

In the following, we look at how inter- and intra-individual variation contribute to ICC differences.

## Generating reliability field maps:
 * For this tutorial we compare functional connectivity from data processed with GSR and without GSR.
 * After importing libraries we load in results generated from our relibility model, most importantly ICC, raw intra-individual variation, raw inter-individual variation which we will use for both reliability field maps and gradient flow maps.
 * Light grey lines from the origin, (0,0), mark ICC values starting from 0.1 (shallowest angled bottom line) to 0.9 (steepest most left) - ICC of 0 and 1 are the X and Y axes respectively.
 * We will first create individual field maps with the function **single_fieldmap()**.
 * Overlaying the individual field maps with just the contour lines allows visual comparison of inter- and intra-individual variation on ICC (**field_map_overlay()**).
 
 
## 1. Individual fieldmaps for GSR and no GSR
 * _tasks_, _taskcolors_, _taskcmaps_ should include all conditions included in _data_
 * _taskcolors_ are individual colors for each task - can be hex or color name as set here
 * _taskcmaps_ should be color maps corresponding to each _task_ and _taskcolor_

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.cm as mcm
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import pandas as pd
import seaborn as sns
from scipy import stats
from seaborn.utils import iqr, _kde_support, remove_na

from six import string_types

# Calculate KDE for field maps:
def _scipy_bivariate_kde(x, y, bw, gridsize, cut, clip):

    """Compute a bivariate kde using scipy."""
    
    data = np.c_[x, y]
    kde = stats.gaussian_kde(data.T, bw_method=bw)
    data_std = data.std(axis=0, ddof=1)
    if isinstance(bw, string_types):
        bw = "scotts" if bw == "scott" else bw
        bw_x = getattr(kde, "%s_factor" % bw)() * data_std[0]
        bw_y = getattr(kde, "%s_factor" % bw)() * data_std[1]
    elif np.isscalar(bw):
        bw_x, bw_y = bw, bw
    else:
        msg = ("Cannot specify a different bandwidth for each dimension "
               "with the scipy backend. You should install statsmodels.")
        raise ValueError(msg)
    x_support = _kde_support(data[:, 0], bw_x, gridsize, cut, clip[0])
    y_support = _kde_support(data[:, 1], bw_y, gridsize, cut, clip[1])
    xx, yy = np.meshgrid(x_support, y_support)
    z = kde([xx.ravel(), yy.ravel()]).reshape(xx.shape)
    return xx, yy, z

# KDE plot function:
def _bivariate_kdeplot(xx1, yy1, z1scale, filled, fill_lowest,
                       kernel, bw, gridsize, cut, clip,
                       axlabel, cbar, cbar_ax, cbar_kws, ax, **kwargs):
    from seaborn.palettes import color_palette, light_palette, dark_palette, blend_palette
    """Plot a joint KDE estimate as a bivariate contour plot."""
    # Determine the clipping
    if clip is None:
        clip = [(-np.inf, np.inf), (-np.inf, np.inf)]
    elif np.ndim(clip) == 1:
        clip = [clip, clip]
    # Plot the contours
    n_levels = kwargs.pop("n_levels", 10)
    scout, = ax.plot([], [])
    default_color = scout.get_color()
    scout.remove()
    color = kwargs.pop("color", default_color)
#     cmap = kwargs.pop("cmap", None)
    cmap = cbar_kws['cmap']
    label = kwargs.pop("label", None)
    kwargs["cmap"] = cmap
    contour_func = ax.contourf if filled else ax.contour
    cset = contour_func(xx1, yy1, z1scale, n_levels, **kwargs)
    if filled and not fill_lowest:
        cset.collections[0].set_alpha(0)
    kwargs["n_levels"] = n_levels
    if cbar:
        cbar_kws = {} if cbar_kws is None else cbar_kws
        ax.figure.colorbar(cset, cbar_ax, ax, **cbar_kws)
    # Label the axes
    if hasattr(xx1, "name") and axlabel:
        ax.set_xlabel(x.name)
    if hasattr(yy1, "name") and axlabel:
        ax.set_ylabel(y.name)
    if label is not None:
        legend_color = cmap(.95) if color is None else color
        if filled:
            ax.fill_between([], [], color=legend_color, label=label)
        else:
            ax.plot([], [], color=legend_color, label=label)
    return ax

def plot_field_map(x,y,taskcolor,taskcmap,alpha,lines,outpath,thr=0.0001,gridsize=300,
                     overlay=False,cbar_option=True,figSize=(12,10),xyLim=95,shade=True,addContourLines=True):
    
    plt.figure(figsize=(figSize[0],figSize[1]))
    sns.set_style('white')
    ax=plt.gca()
    mpl.rcParams['font.weight'] = 'bold'
    mpl.rcParams['font.size'] = 1
    sns.set(font_scale=3)
    
    if int(xyLim):
        xperc = np.percentile(x,xyLim)
        yperc = np.percentile(y,xyLim)
        xy_lim = np.max([xperc,yperc])
        xyVals = (0,xy_lim)
    elif type(xyLim) == 'tuple':
        xyVals = (xyLim[0],xyLim[1])

    ax.axes.set_xlim([xyVals[0],xyVals[1]])
    ax.axes.set_ylim([xyVals[0],xyVals[1]])
    plt.xticks(fontweight='bold',fontsize=20)
    plt.yticks(fontweight='bold',fontsize=20)
    plt.xlabel('Intra-individual Variation',labelpad=20,fontweight='bold',fontsize=20)
    plt.ylabel('Inter-individual Variation',labelpad=20,fontweight='bold',fontsize=20)
    bw='scott'
    gridsize=gridsize
    cut=10
    clip = [(-np.inf, np.inf), (-np.inf, np.inf)]
    legend=True
    cumulative=False
    shade=shade
    shade_lowest=False
    cbar=False
    cbar_ax=None
    filled=True
    fill_lowest=False
    vertical=False
    kernel="gau"

    # Kde distribution:
    xx1, yy1, z1 = _scipy_bivariate_kde(x, y, bw, gridsize, cut, clip)

    # Scaling and normalization so that field maps are comparable:
    scaler = float(1000) # Take this out and just make sure line 225 is float
    z1scale = scaler*z1/np.sum(z1)
    normalized = (z1scale-np.min(z1scale))/(np.max(z1scale)-np.min(z1scale))
    
#     # Find max x,y density:
#     maxXY = np.where(normalized == np.max(normalized))
#     print(maxXY)
    # Reset clip for actual kdeplot:
    clip=None

    # Set colorbar for scaled density plot:
    cbar_kws={'cmap':taskcmap}
    our_cmap = plt.get_cmap(taskcmap)
    cmap_max = 1.00001
    norm = mcolors.Normalize(vmin=0, vmax=cmap_max)    
    proxy_mappable = mpl.cm.ScalarMappable(cmap=our_cmap, norm=norm)
    proxy_mappable.set_array(normalized)   
    ax = _bivariate_kdeplot(xx1, yy1, normalized, shade, 
                            shade_lowest, kernel, bw, gridsize, 
                            cut, clip, legend, cbar, cbar_ax, cbar_kws, 
                            ax,vmin=0,vmax=cmap_max,levels=5,alpha=alpha,
                           linewidths=5)
    if addContourLines == True:
        ax = plt.contour(xx1,yy1,normalized,5,colors = taskcolor)
    if cbar_option == True:
        cbar = plt.colorbar(proxy_mappable, boundaries=np.arange(0,cmap_max,.1), spacing='proportional', orientation='vertical', pad=.01)
        cbar.set_label('Density',labelpad=20)

    if lines == True:
        plt.plot([1,0],[1,0],color='black',alpha=0.3,zorder=0)
        for iccline in [0.2,0.4,0.6,0.8]:
#             plt.plot([1,0],[iccline,0],color='black',alpha=0.3,zorder=len(taskcombos)+1)
#             plt.plot([iccline,0],[1,0],color='black',alpha=0.3,zorder=len(taskcombos)+1) 
            plt.plot([1,0],[iccline,0],color='black',alpha=0.3)
            plt.plot([iccline,0],[1,0],color='black',alpha=0.3) 

    if outpath == True:
        plt.savefig('../figures/shortpaper/fieldmaps/%s_%s_perc%s_fieldmap_nogsr_front_contour_070121_time_1200-600.png' % (taskcombo[0],taskcombo[1],percnum),dpi=300)
    plt.show()   
    return xx1,yy1,normalized



In [ ]:
# Set colormaps/colors for field map contours
tasks = ['REST_nogsr',
         'REST_gsr']
taskcolors = {'REST_gsr':'grey',
              'REST_nogsr':'red'}
taskcmaps = {'REST_gsr':'Greys',
             'REST_nogsr':'Reds'}
outpath = False # Not saving any figures for now
lines = True # Show ICC lines
alpha = 1 
thr = 0.0001 # Our threshold for which variability values to include

# Plot individual field maps:
for num,task in enumerate(['REST_nogsr','REST_gsr']):
    x = data[task]['raww_masked']
    y = data[task]['rawb_masked']
    xx1,yy1,normalized = plot_field_map(x,y,taskcolors[task],taskcmaps[task],alpha,lines,outpath,thr=0.0001,gridsize=50,
                         overlay=False,cbar_option=True,figSize=(12,10),xyLim=95,shade=True,addContourLines=True)
    
    xperc = np.percentile(x,95)
    yperc = np.percentile(y,95)
    xy_lim = np.max([xperc,yperc])
    xyVals = (0,xy_lim)
    # Small bins
    plt.figure(figsize=(10,10))
    plt.hist2d(x, y, bins=(100, 100), cmap=taskcmaps[task])
    plt.xlim([0,xyVals[1]])
    plt.ylim([0,xyVals[1]])
    plt.show()
    
    plt.figure(figsize=(10,10))
    plt.hist2d(x, y, bins=(300, 300), cmap=taskcmaps[task])
    plt.xlim([0,xyVals[1]])
    plt.ylim([0,xyVals[1]])
    plt.show()
    
    plt.figure(figsize=(10,10))
    plt.hist2d(x, y, bins=(500, 500), cmap=taskcmaps[task])
    plt.xlim([0,xyVals[1]])
    plt.ylim([0,xyVals[1]])
    plt.show()

## 2. Field map overlap for GSR and no GSR

In [ ]:
# # Not saving any figures for now.
# outpath = False
# lines = True
# alpha = 1

# # Choosing conditions to compare:
# taskcombos = [['REST_nogsr',
#                'REST_gsr']]

# plot_field_map_overlay(taskcombos,data,taskcolors,taskcmaps,alpha,lines,outpath,
#                       cbar_option=True,figSize=(12,10),xlim=(0,0.025),ylim=(0,0.025),
#                      shade=False,thr=0.0001)

## Generating gradient flow vector angular histograms:
 * Gradient flow vectors are useful understand optimality of change in variation with respect to improved reliability. Gradient flow angular histograms show the frequency of edges for each angle.
 * Gradient flow histograms are created using **gradient_flow_histogram()**.
     * 1. intra- and inter-individual variation is used to calculate standardized gradient flow vectors and the count for each angle is plotted.
     * 2. Each angle is counted and the total number is plotted according to the gradient flow circular colormap.

In [ ]:
# # Paired masking:
# # Input data should be masked such that each intra- and inter-individual variation value are paired between the two conditions
# bothmask = np.intersect1d(data[tasks[0]]['totmask_cortex'],data[tasks[1]]['totmask_cortex'])
# x0 = data[tasks[0]]['raww'][bothmask]
# y0 = data[tasks[0]]['rawb'][bothmask]
# icc0 = data[tasks[0]]['icc'][bothmask]
# x1 = data[tasks[1]]['raww'][bothmask]
# y1 = data[tasks[1]]['rawb'][bothmask]
# icc1 = data[tasks[1]]['icc'][bothmask]

# # Calculate vector parameters:
# df = calc_icc_vectors(x0,y0,x1,y1,
#                       icc0,icc1,tasks[0],tasks[1])
# theta = np.array([convertAngle(df['theta0'][i],df['xdiff'][i]) for i in range(len(df['theta0']))])
# theta = theta[~np.isnan(theta)]

# # Gradient flow histogram plot options:
# bin_threshold = 5 # degrees covered per histogram bar
# outpath = False # No saving for now.
# title = False # No title for now.

# # Plot gradient flow histogram:
# pah(theta,bin_threshold,rvb,title,outpath)


## Generating surface plots for ICC differences and parcel-wise gradient flow vectors:
 * Mean of ICC differences and mean gradient flow vector direction for each parcel connectivity vector (row/column of ICC/gradient flow matrix) are plotted on the cortical surface with **plot_surface_comparisons()** .
 * **calc_icc_vectors_mean()** first calculates standardized gradient flow vectors using the parcel vector mean of inter- and intra-individual differences to yield mean gradient flow vector direction for each parcel.
     * Separate surface plots are generated for positive and negative ICC difference and gradient flow direction for clarity.
     * Due to averaging across the parcel connectivity vector, ICC difference and gradient flow direction might not have complete congruency. 

## GSR vs No GSR surface

Every 4 Surface plots in order shown (Left lateral, left medial, right lateral, right medial:
1. Positive change gradient flow vectors
2. Positive change ICC
3. Negative change gradient flow vectors
4. Negative change ICC


In [ ]:
# #############################################################
# # make positive and negative cmaps instead of changing data #
# #############################################################
# taskcombos = [['REST_gsr','REST_nogsr']]
# outpath = None
# numparcels = 1
# alpha = 1
# glasserlabel,(ax1,ax2) = cifti.read(plabel)
# surfaces = [lsurf,rsurf]
# numparcels = 1
# alpha = 1
# darkness = 0.1
# data_range = (0,360)
# cmap = rvb
# parcellation = glasserlabel
# outpath = False
# plot_surface_comparisons(taskcombos, data, parcellation, surfaces, numparcels, alpha, darkness, data_range,cmap,outpath)



While data processed without GSR has overall greater reliability compared to data processed with GSR, we can see topological differences in different brain regions as shown by surface plots showing ICC differences as well as gradient vectors. With this information different preprocessing steps can be applied when higher reliability is desired for the target region being studied. 

In [ ]:
# import sys
# import cifti
# import matplotlib.pyplot as plt
# sys.path.append('../code')
# from gradient_flow_vectors import calc_icc_vectors_mean, convertAngle,calc_icc_vectors
# lsurf = surfaces[0]
# rsurf = surfaces[1]
# for taskcombo in taskcombos:
#     for posNeg in ['positive','negative']:
#         task1 = taskcombo[0]
#         task2 = taskcombo[1]
#         # Vector angles:
#         mask1 = data[task1]['totmask']
#         mask2 = data[task2]['totmask']
#         bothMask = np.intersect1d(mask1,mask2)
#         icc0 = np.nanmean(array2mat(data[task1]['icc'],447),0)[0]
#         icc1 = np.nanmean(array2mat(data[task2]['icc'],447),0)[0]
#         x0 = np.nanmean(array2mat(data[task1]['raww'],447),0)[0]
#         y0 = np.nanmean(array2mat(data[task1]['rawb'],447),0)[0]
#         x1 = np.nanmean(array2mat(data[task2]['raww'],447),0)[0]
#         y1 = np.nanmean(array2mat(data[task2]['rawb'],447),0)[0]
#         df = calc_icc_vectors(x0,y0,x1,y1,icc0,icc1,task1,task2)

#         plotname =  '%s-%s_%s_vectors' % (task2,task1,posNeg)
#         converted_angles = np.array([convertAngle(df['theta0'][i],df['xdiff'][i]) for i in range(len(df['theta0']))])
#         angVerts = parcel2vert(parcellation,converted_angles)
#         posNegMask = parcel2vert(parcellation,icc1-icc0)
#         meandICC = np.mean(posNegMask,0)
#         numVertices = int(angVerts.shape[1]/2.)
#         symmetric_cmap = False
#         parcellation_mask = np.where(glasserlabel[0,:] == 0)[0]
#         angVerts[0,parcellation_mask] = np.nan

# warm_cmap_whites,cold_cmap_whites = warm_cold_gradient_flow_cmap()
# # testarray = np.ones([1,20484]) * 300
# # testarray[0,parcellation_mask] = 45
# numVertices = int(angVerts.shape[1]/2.)
# symmetric_cmap = False
# plot_surface(angVerts,lsurf,rsurf,numVertices,data_range,warm_cmap_whites,alpha,
#                          darkness,symmetric_cmap,False,outpath,'%s-%s_%s_dICC_angle' % (task2,task1,posNeg))


In [ ]:
# from brainspace.plotting import plot_hemispheres
# from brainspace.mesh.mesh_io import read_surface
# from brainspace.datasets import load_conte69
# %matplotlib inline
# # Conte surface
# lsurf = read_surface('../misc/surfaces/Conte69.L.very_inflated.10k_fs_LR.surf.gii')
# rsurf = read_surface('../misc/surfaces/Conte69.R.very_inflated.10k_fs_LR.surf.gii')
# plot_hemispheres(lsurf, rsurf, array_name=angVerts, size=(800, 200),cmap=warm_cmap_whites,
#                 color_bar=False,interactive=False,embed_nb=True,nan_color=(1,1,1,1))
# plot_hemispheres(lsurf, rsurf, array_name=angVerts, size=(800, 200),cmap=cold_cmap_whites,
#                 color_bar=False,interactive=False,embed_nb=True,nan_color=(1,1,1,1))